###Imports

In [1]:
import keras
from keras.layers import Conv2D, BatchNormalization, MaxPool2D, GlobalMaxPool2D , Flatten
from keras.layers import TimeDistributed, LSTM, Dense, Dropout
import numpy as np
import datetime
import tensorflow as tf
import pandas as pd
import h5py
from keras import backend as k
from keras.callbacks import ModelCheckpoint

#tf.debugging.set_log_device_placement(True)

The convNet:

In [2]:
def video_generator(path , batch_size = 1 , train_data_size = 180 , test_data_size = 16 , mode = "train"):

    count = 1
    #infinite loop
    if(mode == "train"):
        while True:
            #check if it is the end of train data size
            #if its the end, set the counter to 1 for re-iteration
            if(count > train_data_size):
                count = 1
                
            #read hdf5
            file = h5py.File(path + "video (" + str(count) + ").hdf5" , "r+")
            video = np.array(file["/video"]).astype("uint8").reshape((1 , 300 , 360 , 640 , 3))
            label = np.array(file["/label"]).astype("uint8").reshape((1 , 1))

            label = np.log(label)
            
            #print("Now training on example " + str(count))
            count = count + 1
            
            #yield the input
            yield (video , label)

    elif(mode == "eval"):
        while True:
            #check if it is the end of train data size
            #if its the end, set the counter to 1 for re-iteration
            if(count > test_data_size):
                count = 1
                
            #read hdf5
            file = h5py.File(path + "video (" + str(count + train_data_size) + ").hdf5" , "r+")
            video = np.array(file["/video"]).astype("uint8").reshape((1 , 300 , 360 , 640 , 3))
            label = np.array(file["/label"]).astype("uint8").reshape((1 , 1))

            label = np.log(label)

            print("Now testing on example " + str(count))

            count = count + 1
            
            #yield the input
            yield (video , label)

In [3]:
def single_convnet(shape):
    momentum = .9
    model = keras.Sequential()
    #model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(Conv2D(32, (3,3), strides=(2 , 2) , input_shape=shape, padding='valid', activation='relu' , name = "conv_1"))
    model.add(BatchNormalization(momentum=momentum))
    
    model.add(MaxPool2D())
    
    #model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
    #model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
    #model.add(BatchNormalization(momentum=momentum))
    
    #model.add(MaxPool2D())
    
    #model.add(Conv2D(256, (3,3), padding='same', activation='relu'))
    model.add(Conv2D(32, (3,3), strides=(2 , 2) , padding='valid', activation='relu' , name = "conv_2"))
    model.add(BatchNormalization(momentum=momentum))
    
    model.add(MaxPool2D())
    
    #model.add(Conv2D(1024, (3,3), padding='same', activation='relu'))
    model.add(Conv2D(32, (1,1) , strides=(2 , 2) , padding='valid', activation='relu' , name = "conv_3"))
    model.add(BatchNormalization(momentum=momentum))
    model.add(MaxPool2D())
    
    # flatten...
    #model.add(GlobalMaxPool2D())
    model.add(Flatten())
    model.add(Dense((256) , activation="relu"))
    model.summary()
    return model

In [4]:
def Model(shape):
    # Create our convnet with (256 , 144 , 3) input shape
    convnet = single_convnet(shape[1:])
    
    # then create our final model
    model = keras.Sequential()
    # add the convnet with (200 , 256 , 144 , 3) shape
    model.add(TimeDistributed(convnet, input_shape=shape))
    # here, you can also use GRU or LSTM
    model.add(LSTM(32))
    # and finally, we make a decision network
    #model.add(Dense(1024, activation='relu'))
    #model.add(Dropout(.5))
    #model.add(Dense(512, activation='relu'))
    #model.add(Dropout(.5))
    #model.add(Dense(128, activation='relu'))
    #model.add(Dropout(.5))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1))
    return model

In [5]:
#load the already existing model along with weights etc.
#model = keras.models.load_model("model")

#Else compile the model
model = Model((300 , 360 , 640 , 3))
model.summary()
#load weights if there are saved weights
model.load_weights("model/variables/variables")
model.compile(loss=keras.losses.MeanSquaredError(), optimizer='adam', metrics=['mse'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_1 (Conv2D)              (None, 179, 319, 32)      896       
_________________________________________________________________
batch_normalization (BatchNo (None, 179, 319, 32)      128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 89, 159, 32)       0         
_________________________________________________________________
conv_2 (Conv2D)              (None, 44, 79, 32)        9248      
_________________________________________________________________
batch_normalization_1 (Batch (None, 44, 79, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 22, 39, 32)        0         
_________________________________________________________________
conv_3 (Conv2D)              (None, 11, 20, 32)        1

In [8]:
#this cell is for testing only
#do not use in the actual implementation
x = np.ones((1 , 300 , 360 , 640 , 3))
y = np.array([.25])
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)



with tf.device("/CPU:0"):
    model.fit(x , y , epochs = 50 , callbacks=[tensorboard_callback])

Epoch 1/50
1/1 [==============================] - 0s 290ms/step - loss: 2.1354 - mse: 2.1354
Epoch 2/50


In [5]:
#setting up tensorboard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

#setting up checkpoint
checkpoint = ModelCheckpoint("model", monitor='loss', verbose=1, save_best_only=True, mode='auto', save_freq=20)

#setting the path for training and test data
path = "YoutubeScrapper/python/out/compresseddata/"

#Assigning batch size
batch_size = 1

#Assigning training data size
train_data_size = 180

#Initializing the generator
training_data = video_generator(path , batch_size=1 , train_data_size = train_data_size , test_data_size = test_data_size , mode = "train")


with tf.device("/CPU:0"):
    model.fit(x=training_data , steps_per_epoch=train_data_size , epochs=1 , callbacks=[tensorboard_callback , checkpoint])

NotImplementedError: in user code:

    C:\Users\saadb\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\engine\training.py:806 train_function  *
        return step_function(self, iterator)
    C:\Users\saadb\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\engine\training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\saadb\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\saadb\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\saadb\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Us

In [6]:
#Assigning batch size
batch_size = 1

#Assigning training data size
train_data_size = 180

#setting the path for training and test data
path = "YoutubeScrapper/python/out/compresseddata/"

#Assigning test data size
test_data_size = 16

#evaluating data
test_data = video_generator(path , mode = "eval")

with tf.device("/CPU:0"):
    model.evaluate(test_data , steps = test_data_size)

Now testing on example 1
Now testing on example 2
10/16 [=================>............] - ETA: 13s - loss: 0.0686 - mse: 0.0686Now testing on example 11
Now testing on example 12
12/16 [=====================>........] - ETA: 9s - loss: 0.0623 - mse: 0.0623Now testing on example 13
Now testing on example 14
16/16 [==============================] - 37s 2s/step - loss: 0.0859 - mse: 0.0859
